# Regressionsanalys med dummyvariabler

*Av Anders Sundell*

Regressionsanalys kräver att de oberoende variablerna är på intervallskalenivå, dvs att alla skalsteg är lika långa. Eftersom analysen går ut på att dra en rak linje över de olika värdena på den oberoende variabeln blir det konstigt om det inte finns någon konsekvens i de olika skalstegen, eller om de inte är rangordnade.

Variabler som till exempel inte lämpar sig för att lägga in som oberoende variabler i en regressionsanalys är partitillhörighet, födelseort eller personlighetstyp.

Men det finns en lösning: "dummyvariabler". En variabel som bara har två värden har lika långt mellan alla skalsteg (det enda) och kan därför användas i en regressionsanalys. Genom att dela upp kategoriska variabler i flera sådana dummyvariabler, där varje variabel motsvarar ett värde på den ursprungliga variabeln, kan de olika kategorierna analyseras med regressionsanalys.

I praktiken blir det som att jämföra medelvärden i de olika grupperna. Fördelen är att man i regressionsanalysen samtidigt kan kontrollera för andra variabler. I den här guiden går vi igenom vad dummyvariabler är, hur man skapar dem i Stata, och hur man lägger in dem i en regressionsanalys.

## Vad dummyvariabler är

Dummyvariabler är variabler som delar upp en kategorisk variabel i alla sina värden, minus ett. Ett värde utelämnas alltid, som referenskategori. Koefficienterna för de nya delvariablerna kommer sedan visa de förväntade skillnaderna, gentemot referenskategorin. Har man en ursprunglig variabel med fem värden får man fyra dummyvariabler. Har man en variabel med tre värden får man två dummyvariabler.

Dummyvariablerna har värdet 0 eller 1: Har analysenheten värdet ifråga får den värdet 1, annars 0.

Säg till exempel att vi vill se effekten av ett lands valsystem på antalet partier som är representerade i parlamentet. Klassiska teorier säger att i länder med mer majoritära valsystem (där vinnaren tar alla mandat i en valkrets tex) kommer antalet partier att bli lägre, medan länder som har mer proportionella valsystem (tex Sverige, där mandaten fördelas proportionellt) blir antalet partier fler.

I QoG-datamängden finns en variabel för valsystem, `gol_est`. Den har tre värden: `Majoritarian`, `Proportional` och `Mixed`. Man kan kanske hävda att Mixed-kategorin är nåt sorts mittemellan mellan Majoritarian och Proportional, men säkrare är att behandla det som tre olika kategorier. Vi behöver då skapa två dummyvariabler för att kunna analysera variabeln på ett bra sätt. De skapas enligt följande princip:

| Ursprungsvärde | dum_proportional | dum_mixed |
|----------------|------------------|-----------|
| Majoritarian   | 0                | 0         |
| Proportional   | 1                | 0         |
| Mixed          | 0                | 1         |

Majoritarian-kategorin är här referens, och får noll på de två nya variablerna som skapas.
Proportional får en etta på `dum_proportional` och Mixed får en etta på `dum_mixed`. Koefficienterna för `dum_proportional` och `dum_mixed` kommer alltså visa skillnaden mellan de två kategorierna och Majoritarian-kategorin - vill man se skillnaden mellan tex Proportional och Mixed i koefficienterna får man lämna någon av dem som referens.

## Skapa dummyvariabler

Vi ska nu rent konkret göra dummyvariablerna. Vi använder oss av QoG-basic, som kan laddas in direkt i Stata med länken nedan. Den oberoende variabeln heter `gol_est` och har tre värden. Nedan laddar vi in datamängden och gör en frekvenstabell.

In [2]:
use "https://www.qogdata.pol.gu.se/dataarchive/qog_bas_cs_jan18.dta", clear
tab gol_est


(Quality of Government Basic dataset 2018 - Cross-Section)


   Electoral |
      System |
      Type-3 |
     classes |      Freq.     Percent        Cum.
-------------+-----------------------------------
Majoritarian |         49       37.98       37.98
Proportional |         59       45.74       83.72
       Mixed |         21       16.28      100.00
-------------+-----------------------------------
       Total |        129      100.00


Nu är det dags att göra variablerna. Det finns flera olika sätt. Det lite krångligare sättet är att göra egna variabler med `generate` och sedan ge dem rätt värden med `replace`. Men smartare är att använda sig av `tab`, med ett option som skapar dummyvariabler `, generate()`.

Vi skriver då namnet på variabeln vi vill göra dummyvariabler av, lägger till option `, generate()` och skriver inom parenteserna vad vi vill att dummyvariablerna ska heta. Vi kallar dem dum_valsystem för att indikera att det är dummyvariabler och att det handlar om valsystemet.

In [7]:
tab gol_est, generate(dum_valsystem)


   Electoral |
      System |
      Type-3 |
     classes |      Freq.     Percent        Cum.
-------------+-----------------------------------
Majoritarian |         49       37.98       37.98
Proportional |         59       45.74       83.72
       Mixed |         21       16.28      100.00
-------------+-----------------------------------
       Total |        129      100.00


Om man nu kollar i variabellistan ser man att det dykt upp tre ny variabler: `dum_valsystem1`, `dum_valsystem2`och `dum_valsystem3`. Om vi till exempel kollar hur den ursprungliga `gol_est` förhåller sig till `dum_valsystem1` så ser vi följande:

In [9]:
tab gol_est dum_valsystem1


   Electoral |
      System |
      Type-3 | gol_est==Majoritarian
     classes |         0          1 |     Total
-------------+----------------------+----------
Majoritarian |         0         49 |        49 
Proportional |        59          0 |        59 
       Mixed |        21          0 |        21 
-------------+----------------------+----------
       Total |        80         49 |       129 


Alla analysenheterna som hade värdet "Majoritarian" har fått en 1:a på den nya variabeln, och de som hade antingen "Proportional" eller "Mixed" har fått värdet 0, precis som det var tänkt. Nu kan vi använda de här variablerna i analysen.

## Regressionsanalys med dummyvariabler

Den beroende variabeln som vi ska använda heter `gol_enep`, och är ett mått på antalet "effektiva partier", dvs antalet partier i parlamentet med någon reell möjlighet att påverka. För att illustrera vad det är regressionsanalysen kommer att visa kan vi först titta på den beroende variabelns medelvärde i de olika grupperna (se separat guide för närmare förklaring).

In [11]:
table gol_est, contents(mean gol_enep)


-----------------------------
Electoral    |
System       |
Type-3       |
classes      | mean(gol_enep)
-------------+---------------
Majoritarian |      3.5251438
Proportional |      4.7289557
       Mixed |      3.7276943
-----------------------------


Vi ser här att i länder med majoritärt valsystem är det i snitt 3.5 (effektiva) partier, där det är proportionella valsystem är det 4.7, och där det är blandat är det 3.7. 

Regressionsanalysen kommer egentligen inte visa något annorlunda. Fördelen är att vi då kan se om skillnaderna är signifikanta, och att vi dessutom kan kontrollera för eventuella bakomliggande variabler.

Nu gör vi regressionsanalysen, med `gol_enep` som beroende variabel, och `dum_valsystem2` och `dum_valsystem3`som oberoende variabler. Vi utelämnar alltså `dum_valsystem1` - det blir vår referenskategori.

In [12]:
reg gol_enep dum_valsystem2 dum_valsystem3


      Source |       SS           df       MS      Number of obs   =       120
-------------+----------------------------------   F(2, 117)       =      3.97
       Model |  39.1875707         2  19.5937854   Prob > F        =    0.0214
    Residual |  577.131481       117   4.9327477   R-squared       =    0.0636
-------------+----------------------------------   Adj R-squared   =    0.0476
       Total |  616.319052       119  5.17915169   Root MSE        =     2.221

--------------------------------------------------------------------------------
      gol_enep |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
---------------+----------------------------------------------------------------
dum_valsystem2 |   1.203812   .4531648     2.66   0.009     .3063428    2.101281
dum_valsystem3 |   .2025504   .5959897     0.34   0.735    -.9777758    1.382877
         _cons |   3.525144   .3468586    10.16   0.000     2.838208    4.212079
--------------------------------------

`_cons` visar det förväntade värdet på den beroende variabeln när alla de oberoende variablerna är 0. Dvs, landet har noll på `dum_valsystem2` och `dum_valsystem3`, dvs, landet har ett majoritärt valsystem. Värdet är 3.5, alltså medelvärdet för majoritära valsystem.

Koefficienten för `dum_valsystem2` är 1.2, och visar alltså skillnaden mellan de proportionella valsystemen och de majoritära. Adderar vi 1.2 till 3.5 får vi 4.7, alltså medelvärdet för de proportionella valsystemen. Vi ser också att den skillnaden är signifikant (p=0.009).

Däremot är skillnaden mellan de mixade valsystemen `dum_valsystem3`och de majoritära mindre (0.2) och den skillnaden är inte heller signifikant (p=0.735).

Men är skillnaden mellan de proportionella och de mixade valsystemen signifikant? Det ser  vi inte här. För att ta reda på det kan vi istället utelämna `dum_valsystem2`, dvs vi låter de proportionella valsystemen vara referenskategori.

In [13]:
reg gol_enep dum_valsystem1 dum_valsystem3


      Source |       SS           df       MS      Number of obs   =       120
-------------+----------------------------------   F(2, 117)       =      3.97
       Model |  39.1875707         2  19.5937854   Prob > F        =    0.0214
    Residual |  577.131481       117   4.9327477   R-squared       =    0.0636
-------------+----------------------------------   Adj R-squared   =    0.0476
       Total |  616.319052       119  5.17915169   Root MSE        =     2.221

--------------------------------------------------------------------------------
      gol_enep |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
---------------+----------------------------------------------------------------
dum_valsystem1 |  -1.203812   .4531648    -2.66   0.009    -2.101281   -.3063428
dum_valsystem3 |  -1.001261   .5656325    -1.77   0.079    -2.121467    .1189441
         _cons |   4.728956   .2916288    16.22   0.000       4.1514    5.306511
--------------------------------------

Koefficienten för `dum_valsystem1` är -1.2, dvs exakt samma som den för `dum_valsystem2` i den föregående analysen, fast negativ. Skillnaden mellan de två grupperna är densamma oavsett vilken grupp vi har som referenskategori. Majoritära valsystem har 1.2 partier *färre* än de proportionella; proportionella valsystem har 1.2 parier *fler* än de majoritära.

Men oavsett vilken kategori vi väljer som referens är modellen egentligen densamma. Det kan vi se bland annat på $R^2$-värdet, som är desamma i de två analyserna.

## Regressionsanalyser med automatiskt kodade dummyvariabler

Slutligen kan vi när vi gör en regressionsanalys låta Stata göra dummyvariablerna åt oss, i själva regressionskommandot. Vi behöver alltså inte göra steget när vi skapar `dum_valsystem1` osv. Istället anger vi bara ursprungsvariabeln `gol_est` som oberoende variabel, men skriver ett b1. framför. Alltså `b1.gol_est`. Stata gör då temporära dummyvariabler, som inte sparas i datasetet efter analysen, och värdet 1 används som referenskategori. Såhär:

In [11]:
reg gol_enep b1.gol_est


      Source |       SS           df       MS      Number of obs   =       120
-------------+----------------------------------   F(2, 117)       =      3.97
       Model |  39.1875707         2  19.5937854   Prob > F        =    0.0214
    Residual |  577.131481       117   4.9327477   R-squared       =    0.0636
-------------+----------------------------------   Adj R-squared   =    0.0476
       Total |  616.319052       119  5.17915169   Root MSE        =     2.221

-------------------------------------------------------------------------------
     gol_enep |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
      gol_est |
Proportional  |   1.203812   .4531648     2.66   0.009     .3063428    2.101281
       Mixed  |   .2025504   .5959897     0.34   0.735    -.9777758    1.382877
              |
        _cons |   3.525144   .3468586    10.16   0.000     2.838208    4.212079
------------

Siffrorna är som synes helt identiska med dem i tabellen ovan där Majoritarian var referenskategori. Vill man ha en annan referenskategori här kan man skriva en annan siffra efter b:et. Här nedan är det med proportionellt valsystem som referens.

In [12]:
reg gol_enep b2.gol_est


      Source |       SS           df       MS      Number of obs   =       120
-------------+----------------------------------   F(2, 117)       =      3.97
       Model |  39.1875707         2  19.5937854   Prob > F        =    0.0214
    Residual |  577.131481       117   4.9327477   R-squared       =    0.0636
-------------+----------------------------------   Adj R-squared   =    0.0476
       Total |  616.319052       119  5.17915169   Root MSE        =     2.221

-------------------------------------------------------------------------------
     gol_enep |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
      gol_est |
Majoritarian  |  -1.203812   .4531648    -2.66   0.009    -2.101281   -.3063428
       Mixed  |  -1.001261   .5656325    -1.77   0.079    -2.121467    .1189441
              |
        _cons |   4.728956   .2916288    16.22   0.000       4.1514    5.306511
------------

## Avslutning

Dummyvariabler är nödvändiga för att kunna använda kategoriska variabler utan rangordning som oberoende variabler i regressionsanalys. Man måste dock inte nödvändigtvis ge varje enskilt värde en egen dummyvariabler - det går lika bra att välja ut några enskilda värden som jämförs med resten. Det viktiga är att man kommer ihåg vad det är för jämförelse som faktiskt görs. Alla värden som inte får en egen variabel är referensen, och dummyvariablerna visar skillnaderna gentemot just den kategorin.